In [1]:
class Parent():
    def __init__(self, a=1, b=2, c=3):
        self.a=a
        self.b=b
        self.c=c
    def _print(self, s):
        print(f"I`m parent: a={self.a}\tb={self.b}\tc={self.c}\n{s}")
    
    def __call__(self, s):
        self._print(s)

class Child(Parent):
    def _print(self, s):
        print(f"I`m child: a={self.a}\tb={self.b}\tc={self.c}\n{s}")


In [2]:
child = Child(a=11)
child("woca")

I`m child: a=11	b=2	c=3
woca


In [1]:
import torch
import torch.nn as nn
from torchinfo import summary
class CoupledHead(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        self.layer = nn.Conv2d(in_c, out_c, 3, 1, 1)
    def forward(self, x):
        return self.layer(x)


class DecoupledHead(nn.Module):
    def __init__(self, in_c, out_c, split_ratio):
        super().__init__()
        out_c_1 = int(out_c * split_ratio)
        out_c_2 = out_c - out_c_1
        self.layer1 = nn.Conv2d(in_c, out_c_1, 3, 1, 1)
        self.layer2 = nn.Conv2d(in_c, out_c_2, 3, 1, 1)
    def forward(self, x):
        x1 = self.layer1(x)
        x2 = self.layer2(x)
        out = torch.cat([x1, x2], dim=1)
        return out


In [7]:
# img = torch.randn(2,64,5120, 5120)
m1 = CoupledHead(64, 64)
summary(m1, [2, 64, 2000, 2000])

Layer (type:depth-idx)                   Output Shape              Param #
CoupledHead                              --                        --
├─Conv2d: 1-1                            [2, 64, 2000, 2000]       36,928
Total params: 36,928
Trainable params: 36,928
Non-trainable params: 0
Total mult-adds (G): 295.42
Input size (MB): 2048.00
Forward/backward pass size (MB): 4096.00
Params size (MB): 0.15
Estimated Total Size (MB): 6144.15

In [8]:
m2 = DecoupledHead(64, 64, 0.3)
summary(m2, [2, 64, 1000, 1000])

RuntimeError: CUDA out of memory. Tried to allocate 490.00 MiB (GPU 0; 10.92 GiB total capacity; 5.39 GiB already allocated; 183.00 MiB free; 5.40 GiB reserved in total by PyTorch)

In [3]:
import math
def fmxdy(x, p):
    y = math.pow(x, p)
    return x % p == y % p

x = 12
p = 5
fmxdy(x, p)

True

In [5]:
def qpow(x, n):
    if n == 0:
        return 1
    elif n % 2 == 0:
        return qpow(x, n / 2) ** 2
    else:
        return qpow(x, n - 1) * x

qpow(2, 32)

4294967296

In [2]:
import cv2
import numpy as np
img = cv2.imread('data/widerface/WIDER_train/images/0--Parade/0_Parade_marchingband_1_5.jpg')

In [4]:
img_float32 = img.astype(np.float32).copy()
img_resize = cv2.resize(img_float32, (640, 640))
img_float32.dtype

dtype('float32')

In [7]:
import time
epoch = 1100
val1, val2 = 0, 0
for i in range(epoch):
    t0 = time.time()
    img_resize = cv2.resize(img_float32, (640, 640))
    t1 = time.time()
    img_int8 = cv2.resize(img, (640, 640))
    val2 += time.time() - t1
    val1 += t1 - t0

    
print(val1, val2)

1.6409554481506348 0.7849476337432861
